# 1st-level Analysis

This notebook performes the 1st-level analysis in subject space by executing the following steps:

    1. Aggregate 1st-level model parameters
    2. Specify 1st-level contrasts
    3. Estimate 1st-level contrasts
    4. Normalization to template space (optional)

The notebook runs on all functional runs of a particular task, computes the specified beta-contrast and normalizes them into template space. If requested, it will also compute one beta contrast per stimuli occurence (usefull for machine learning approaches). Confounding factors and outlier volumes can be added as nuisance regressors in the GLM.

**Note:** This notebook requires that the functional preprocessing pipeline was already executed and that it's output can be found in the dataset folder under `dataset/derivatives/fmriflows/preproc_func`. 

## Data Structure Requirements

The data structure to run this notebook should be according to the BIDS format:

    dataset
    ├── analysis-func_specs.json
    ├── sub-{sub_id}
    │   └── func
    │       └── sub-{sub_id}_*task-{task_id}_run-{run_id}_events.tsv
    └── derivatives
        └── fmriflows
            ├── preproc_anat
            │   └── sub-{sub_id}
            │       └── {sub_id}*_transformComposite.h5
            └── preproc_func
                └── sub-{sub_id}
                    ├── sub-{sub_id}*_task-{task_id}_run-{run_id}_confounds.tsv
                    ├── sub-{sub_id}*_task-{task_id}_run-{run_id}_nss.txt
                    ├── sub-{sub_id}*_task-{task_id}_run-{run_id}_outliers.txt
                    └── sub-{sub_id}*_task-{task_id}_run-{run_id}_tFilter_*_sFilter_*.nii.gz

## Execution Specifications

This notebook will extract the relevant processing specifications from the `analysis-1stlevel_specs.json` file in the dataset folder. In the current setup, they are as follows:

In [ ]:
import json
from os.path import join as opj

spec_file = opj('/data', 'analysis-1stlevel_specs.json')

with open(spec_file) as f:
    specs = json.load(f)

specs

If you'd like to change any of those values manually, overwrite them below:

In [ ]:
# List of subject names
subject_list = specs['subject_list']

# Name of task
task_id = specs['task_id']

# Analysis postfix to use for naming the output folder
postfix = specs['analysis_postfix']

# Mode and width of spatial filter used during preprocessing,
# i.e. Low-Pass, fwhm of 6mm = ['LP', 6]
filters_spatial = specs['filters_spatial']

# High and low-pass filter used during preprocessing
filters_temporal = specs['filters_temporal']

# Nuisance regressors to use in GLM
nuisance_regressors = specs['nuisance_regressors']

# If outliers detected during functional preprocing should be used in GLM
use_outliers = specs['use_outliers']

# Serial Correlation Model to use
model_serial_correlations = specs['model_serial_correlations']

# Model bases to use
model_bases = {specs['model_bases']['name']: {'derivs': specs['model_bases']['derivs']}}

# Estimation Method to use
estimation_method = {specs['estimation_method']['name']: specs['estimation_method']['value']}

# If contrasts should be normalized to template space
normlaize = specs['normalize']

# If contrasts should be computed per run
con_per_run = specs['con_per_run']

# Voxel resolution after normalization
norm_res = specs['norm_res']

# Number of cores to use
n_proc = specs['n_parallel_jobs']

In [ ]:
# Get TR value
import json
import numpy as np
from os.path import join as opj

func_desc = opj('/data', 'task-%s_bold.json' % task_id)

with open(func_desc) as f:
    func_desc = json.load(f)

# Read out relevant parameters
TR = func_desc['RepetitionTime']
TR

Contrasts that should be computed according JSON file:

In [ ]:
# Name of contrasts
condition_names = specs['condition_names']
condition_names

In [ ]:
# Aggregate all user specified contrasts
con_specs = specs['contrasts']
contrast_list = [[c['name'], c['type'], [['T_con_%04d' % i, 'T', condition_names, f] for i, f in enumerate(c['weights'])]] for c in con_specs if c['type'] == 'F']
contrast_list += [[c['name'], c['type'], condition_names, c['weights']] for c in con_specs if c['type'] == 'T']
contrast_list

# Create the Workflow

## Import Modules

In [ ]:
from os.path import join as opj
from nipype import Node, MapNode, Workflow
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.algorithms.misc import Gunzip
from nipype.algorithms.modelgen import SpecifySPMModel
from nipype.interfaces.spm import Level1Design, EstimateModel, EstimateContrast
from nipype.interfaces.ants import ApplyTransforms
from nipype.interfaces.utility import Merge
from nipype.interfaces.io import SelectFiles, DataSink

In [ ]:
# Specify SPM location
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('/opt/spm12-dev/spm12_mcr/spm/spm12')

## Relevant Execution Variables

In [ ]:
# Folder paths and names
exp_dir = '/data/derivatives'
out_dir = 'fmriflows'
work_dir = '/workingdir'

## Implement Nodes

In [ ]:
# Specify 1st-level model parameters (stimuli onsets, duration, etc.)
def subjectinfo(subject_id, task_id):

    import numpy as np
    import pandas as pd
    from glob import glob
    from nipype.interfaces.base import Bunch
    
    # Collect names of event files
    file_template = '/data/sub-%s/func/' % subject_id
    file_template += 'sub-%s_task-%s_run-*_events.tsv' % (subject_id, task_id)
    event_files = sorted(glob(file_template))
    
    # Collect names of non-steady state files
    file_template = '/data/derivatives/fmriflows/preproc_func/sub-%s/' % subject_id
    file_template += 'sub-%s_task-%s_run-*_nss.txt' % (subject_id, task_id)
    nss_files = sorted(glob(file_template))
    
    subject_info = []
    stimuli_order = []
    
    for i, f in enumerate(event_files):
    
        trialinfo = pd.read_table(f)
        stimuli_order.append(list(trialinfo.condition))
        nss = np.loadtxt(nss_files[i])
        conditions = []
        onsets = []
        durations = []
        
        for group in trialinfo.groupby('condition'):
            if group[0] != 'empty':
                conditions.append(str(group[0]))
                onsets.append(list(group[1].onset - nss))
                durations.append(group[1].duration.tolist())

        subject_info.append(Bunch(conditions=conditions,
                                  onsets=onsets,
                                  durations=durations))
   
    return subject_info, stimuli_order

# Get Subject Info - get subject specific condition information
getsubjectinfo = Node(Function(input_names=['subject_id', 'task_id'],
                               output_names=['subject_info', 'stimuli_order'],
                               function=subjectinfo),
                      name='getsubjectinfo')
getsubjectinfo.inputs.task_id = task_id

In [ ]:
# Gunzip NIfTI files for SPM
gunzip = MapNode(Gunzip(), name='gunzip', iterfield=['in_file'])

In [ ]:
# Create SPM model
modelspec = Node(SpecifySPMModel(concatenate_runs=False,
                                 input_units='secs',
                                 output_units='secs',
                                 time_repetition=TR,
                                 high_pass_filter_cutoff=filters_temporal[0][1]),
                 name="modelspec")

In [ ]:
# Create 1st-level desing
level1design = Node(Level1Design(bases=model_bases,
                                 timing_units='secs',
                                 interscan_interval=TR,
                                 model_serial_correlations=model_serial_correlations),
                    name="level1design")

In [ ]:
# Estimate 1st-level model
level1estimate = Node(EstimateModel(estimation_method=estimation_method),
                      name="level1estimate")

In [ ]:
# Estimate 1st-level contrasts
level1conest = Node(EstimateContrast(contrasts=contrast_list),
                    name="level1conest")

In [ ]:
# Merge node
merge = Node(Merge(2, ravel_inputs=True), name='merge')

In [ ]:
# Creation of template brain with desired voxel resolution
template_dir = '/templates/mni_icbm152_nlin_asym_09c/'
brain_template = opj(template_dir, '1.0mm_brain.nii.gz')

# Resample template brain to desired resolution
from nibabel import load, Nifti1Image
from nilearn.image import resample_img
from nibabel.spaces import vox2out_vox

img = load(brain_template)
target_shape, target_affine = vox2out_vox(img, voxel_sizes=norm_res)
img_resample = resample_img(img, target_affine, target_shape, clip=True)
norm_template = opj(template_dir, 'template_brain_%s.nii.gz' %'_'.join([str(n) for n in norm_res]))
img_resample.to_filename(norm_template)

# Normalize contrasts if requested
normfunc = MapNode(ApplyTransforms(reference_image=norm_template,
                                   input_image_type=3,
                                   float=True,
                                   interpolation='LanczosWindowedSinc',
                                   invert_transform_flags=[False],
                                   out_postfix='_norm'),
                   name='normfunc', iterfield=['input_image'])

In [ ]:
# Gzip normalized contrasts
def gzip_contrast(contrast):

    from nibabel import load
    out_file = contrast.replace('.nii', '.nii.gz')
    load(contrast).to_filename(out_file)
    return out_file

gzip = MapNode(Function(input_names=['contrast'],
                        output_names=['out_file'],
                        function=gzip_contrast),
               name='gzip', iterfield=['contrast'])

In [ ]:
# Create nuisance regressors
def create_nuisance_regressors(confounds, nuisance_regressors):

    import numpy as np
    import pandas as pd
    import tempfile

    # To store regressor files into
    regressor_files = []

    # Go through confound files
    for i, c in enumerate(confounds):
        df = pd.read_table(c)
        selection = [k for k in df.keys() for n in nuisance_regressors if n in k]
        dfs = df[selection]
        out_file = tempfile.mkdtemp() + '/confounds_%02d.rst' % i
        np.savetxt(out_file, dfs.values)
        regressor_files.append(out_file)

    return regressor_files

nuisance_reg = Node(Function(input_names=['confounds', 'nuisance_regressors'],
                             output_names=['confounds'],
                             function=create_nuisance_regressors),
                      name='nuisance_reg')
nuisance_reg.inputs.nuisance_regressors = nuisance_regressors

In [ ]:
# Plots design matrix
def plot_design_matrix(SPM):

    import numpy as np
    from matplotlib import pyplot as plt
    from scipy.io import loadmat

    # Using scipy's loadmat function we can access SPM.mat
    spmmat = loadmat(SPM, struct_as_record=False)
    
    # Now we can load the design matrix and the names of the rows
    designMatrix = spmmat['SPM'][0][0].xX[0][0].X
    names = [i[0] for i in spmmat['SPM'][0][0].xX[0][0].name[0]]

    # Value normalization for better visualization
    normed_design = designMatrix / np.abs(designMatrix).max(axis=0)

    # Plotting of the design matrix
    fig, ax = plt.subplots(figsize=(8, 8))
    plt.imshow(normed_design, aspect='auto', cmap='gray', interpolation='nearest')
    ax.set_ylabel('Volume id')
    ax.set_xticks(np.arange(len(names)))
    ax.set_xticklabels(names, rotation=90)
    design_matrix = SPM.replace('.mat', '.svg')
    fig.savefig(design_matrix)
    
    return design_matrix

# Extracts design matrix from SPM.mat and plots it
plot_GLM = Node(Function(input_names=['SPM'],
                               output_names=['out_file'],
                               function=plot_design_matrix),
                      name='plot_GLM')

In [ ]:
# Plots design matrix
def plot_contrasts(contrast, template, contrast_names):

    import numpy as np
    from nibabel import load
    from os.path import split
    from nilearn.plotting import plot_stat_map
    
    # Compute 33% percentile for image thresholding
    data = np.abs(load(contrast).get_data())
    threshold = np.percentile(data[data!=0], 33)

    # Create figure
    title = contrast_names[int(split(contrast)[-1][4:8]) - 1]
    out_file = contrast.replace('.nii.gz', '.svg')
    plot_stat_map(contrast, bg_img=template, display_mode='ortho', title=title,
                  threshold=threshold, symmetric_cbar=False, annotate=False,
                  draw_cross=False, black_bg=True, output_file=out_file)
    return out_file

# Extracts design matrix from SPM.mat and plots it
plot_norm_contrasts = MapNode(Function(input_names=['contrast', 'template',
                                                    'contrast_names'],
                               output_names=['out_file'],
                               function=plot_contrasts),
                      name='plot_contrasts', iterfield=['contrast'])
plot_norm_contrasts.inputs.template = '/templates/mni_icbm152_nlin_asym_09c/1.0mm_T1.nii.gz'
plot_norm_contrasts.inputs.contrast_names = [c[0] for c in contrast_list]

In [ ]:
# Create contrast list for condition per run
def get_contrast_per_run(stimuli_order, condition_names):

    import numpy as np

    # Aggregate event information
    event_list = []
    for i, l in enumerate(stimuli_order):
        event_list.append([z for z in zip(np.full(len(l), i), l)])
    event_info = np.reshape(event_list, (-1, 2))

    unique_contrasts = np.unique(event_info[:,1])
    n_runs = np.unique(event_info[:,0])

    # Create list of contrasts for each condition per run
    contrast_list_run = []
    n_contrasts = len(condition_names)
    n_conditions = len(n_runs)
    condition_labels = []

    for j in range(n_conditions):
        for i in range(n_contrasts):
            name = 'cont_%05d' % (1 + i + j * n_conditions)
            con_id = np.zeros(n_contrasts).tolist()
            run_id = np.zeros(n_conditions).tolist()
            con_id[i] = 1
            run_id[j] = 1
            contrast_list_run.append([
                name, 'T', condition_names, con_id, run_id])
            condition_labels.append(condition_names[i])
    
    return contrast_list_run, condition_labels

# Extracts design matrix from SPM.mat and plots it
contrast_per_run = Node(Function(input_names=['stimuli_order', 'condition_names'],
                                     output_names=['run_contrasts', 'condition_labels'],
                                     function=get_contrast_per_run),
                            name='contrast_per_run')
contrast_per_run.inputs.condition_names = condition_names

## Specify Input & Output Stream

In [ ]:
# Iterate over subject and session id
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name='infosource')
infosource.iterables = [('subject_id', subject_list)]

In [ ]:
# Compute Brain Mask and Extract Brain
def create_file_path(sub_id, task_id, tFilter, sFilter):

    from os.path import join
    from glob import glob
    from os.path import join as opj
    
    # Path to preprocessing folders
    path_anat = '/data/derivatives/fmriflows/preproc_anat/'
    path_func = '/data/derivatives/fmriflows/preproc_func/'

    # tFilter Id
    t_id = '.'.join([str(t) for t in tFilter])
    s_id = '_'.join([str(t) for t in sFilter]) + 'mm'
    
    transforms = glob(opj(path_anat, 'sub-%s' % sub_id,
                          'sub-%s_transformComposite.h5' % sub_id))[0]
    func = sorted(glob(opj(path_func, 'sub-%s' % sub_id,
                           'sub-%s_task-%s_run-*tFilter_%s_sFilter_%s.nii.gz' % (sub_id, task_id, t_id, s_id))))
    outliers = sorted(glob(opj(path_func, 'sub-%s' % sub_id,
                               'sub-%s_task-%s_run-*outliers.txt' % (sub_id, task_id))))
    confounds = sorted(glob(opj(path_func, 'sub-%s' % sub_id,
                                'sub-%s_task-%s_run-*confounds.tsv' % (sub_id, task_id))))
    
    return transforms, func, outliers, confounds

selectfiles = Node(Function(input_names=['sub_id','task_id','tFilter','sFilter'],
                            output_names=['transforms', 'func', 'outliers', 'confounds'],
                            function=create_file_path),
                   name='selectfiles')
selectfiles.iterables = [('tFilter', filters_temporal),
                         ('sFilter', filters_spatial)]
selectfiles.inputs.task_id = task_id

In [ ]:
# Save relevant outputs in a datasink
datasink = Node(DataSink(base_directory=exp_dir,
                         container=out_dir),
                name='datasink')

In [ ]:
# Apply the following naming substitutions for the datasink
substitutions = [('_subject_id_%s/_' % sub,
                  '%s/sub-%s/sub-%s_' % (postfix, sub, sub))
                 for sub in subject_list]
substitutions += [('sub-%s_sFilter_%s_tFilter_%s/' % (sub, s, t),
                   'sub-%s_sFilter_%s_tFilter_%s_' % (sub, s, t))
                  for sub in subject_list
                  for t in ['.'.join([str(t) for t in tFilter]) for tFilter in filters_temporal]
                  for s in ['.'.join([str(t) for t in sFilter]) for sFilter in filters_spatial]]
substitutions += [('_normfunc%d/' % c, '') for c in range(len(contrast_list))]
substitutions += [('_normfunc_run%d/' % c, '') for c in range(1000)]
substitutions += [('multivariate/%s' % postfix, '%s/multivariate' % postfix)]
substitutions += [('univariate/%s' % postfix, '%s/univariate' % postfix)]
datasink.inputs.substitutions = substitutions

## Create 1st-Level Analysis Workflow

In [ ]:
# Create anatomical preprocessing workflow
analysis_1st = Workflow(name='analysis_1st')
analysis_1st.base_dir = work_dir
output_folder = 'analysis_1stLevel'

In [ ]:
# Add nodes to workflow and connect them
analysis_1st.connect([(infosource, selectfiles, [('subject_id', 'sub_id')]),
                      (infosource, getsubjectinfo, [('subject_id', 'subject_id')]),
                      (getsubjectinfo, modelspec, [('subject_info', 'subject_info')]),
                      (selectfiles, gunzip, [('func', 'in_file')]),
                      (gunzip, modelspec, [('out_file', 'functional_runs')]),
                      (modelspec, level1design, [('session_info', 'session_info')]),
                      (level1design, level1estimate, [('spm_mat_file', 'spm_mat_file')]),
                      (level1estimate, level1conest, [('spm_mat_file', 'spm_mat_file'),
                                                      ('beta_images', 'beta_images'),
                                                      ('residual_image', 'residual_image')]),
                      (selectfiles, nuisance_reg, [('confounds', 'confounds')]),
                      (nuisance_reg, modelspec, [('confounds', 'realignment_parameters')]),

                      # Store main results in datasink
                      (level1conest, datasink, [('spm_mat_file', '%s.univariate.@spm_mat' % output_folder),
                                                ('con_images', '%s.univariate.@con' % output_folder),
                                                ('ess_images', '%s.univariate.@ess' % output_folder),
                                                ('spmT_images', '%s.univariate.@spmT' % output_folder),
                                                ('spmF_images', '%s.univariate.@spmF' % output_folder)]),

                      # Create visual outputs and report
                      (level1conest, plot_GLM, [('spm_mat_file', 'SPM')]),
                      (plot_GLM, datasink, [('out_file', '%s.univariate.@spm_mat_svg' % output_folder)]),
                      ])

In [ ]:
# Add outlier parameters if requested by user
if use_outliers:
    analysis_1st.connect([(selectfiles, modelspec, [('outliers', 'outlier_files')])])

In [ ]:
# Normalize contrasts to template space if requested by user
if normlaize:
    analysis_1st.connect([(level1conest, merge, [('con_images', 'in1'),
                                                 ('ess_images', 'in2')]),
                          (merge, normfunc, [('out', 'input_image')]),
                          (selectfiles, normfunc, [('transforms', 'transforms')]),
                          (normfunc, gzip, [('output_image', 'contrast')]),
                          (gzip, plot_norm_contrasts, [('out_file', 'contrast')]),
                          
                          (gzip, datasink, [('out_file', '%s.univariate.@norm_files' % output_folder)]),
                          (plot_norm_contrasts, datasink, [('out_file',
                                                            '%s.univariate.@norm_plot' % output_folder)]),
                         ])

In [ ]:
# Create workflow if contrasts per condition per run should be computed
if con_per_run:
    
    # Estimate 1st-level contrasts - one for each session
    level1conest_run = Node(EstimateContrast(), name="level1conest_run")
    
    # Normalize contrasts
    normfunc_run = normfunc.clone('normfunc_run')
    
    # Gzip contrasts
    gzip_run = gzip.clone('gzip_run')
    
    # Write label file
    def write_labels_file(condition_labels, spm_mat_file):

        import numpy as np
        label_file = spm_mat_file.replace('SPM.mat', 'labels.csv')
        np.savetxt(label_file, condition_labels, fmt='%s')

        return label_file
    
    write_labels = Node(Function(input_names=['condition_labels', 'spm_mat_file'],
                                     output_names=['labels_file'],
                                     function=write_labels_file),
                            name='write_labels')

    # Connect all nodes in this part of the workflow
    analysis_1st.connect([(getsubjectinfo, contrast_per_run, [('stimuli_order', 'stimuli_order')]),
                          (contrast_per_run, level1conest_run, [('run_contrasts', 'contrasts')]),
                          (level1estimate, level1conest_run, [('spm_mat_file', 'spm_mat_file'),
                                                              ('beta_images', 'beta_images'),
                                                              ('residual_image', 'residual_image')]),
                          (level1conest_run, normfunc_run, [('con_images', 'input_image')]),
                          (selectfiles, normfunc_run, [('transforms', 'transforms')]),
                          (normfunc_run, gzip_run, [('output_image', 'contrast')]),
                          (gzip_run, datasink, [('out_file', '%s.multivariate.@norm_files' % output_folder)]),
                          (level1estimate, write_labels, [('spm_mat_file', 'spm_mat_file')]),
                          (contrast_per_run, write_labels, [('condition_labels', 'condition_labels')]),
                          (write_labels, datasink, [('labels_file', '%s.multivariate.@norm_labels' % output_folder)]),
                         ])

## Visualize Workflow

In [ ]:
# Create preproc_anat output graph
analysis_1st.write_graph(graph2use='colored', format='svg', simple_form=True)

# Visualize the graph in the notebook
from IPython.display import SVG
SVG(filename=opj(analysis_1st.base_dir, 'analysis_1st', 'graph.svg'))

# Run Workflow

In [ ]:
# Run the workflow in parallel mode
analysis_1st.run(plugin='MultiProc', plugin_args={'n_procs' : n_proc})

In [ ]:
# Save workflow graph visualizations in datasink
analysis_1st.write_graph(graph2use='flat', format='svg', simple_form=True)
analysis_1st.write_graph(graph2use='colored', format='svg', simple_form=True)

from shutil import copyfile
copyfile(opj(analysis_1st.base_dir, 'analysis_1st', 'graph.svg'),
         opj(exp_dir, out_dir,  output_folder, postfix, 'graph.svg'))
copyfile(opj(analysis_1st.base_dir, 'analysis_1st', 'graph_detailed.svg'),
         opj(exp_dir, out_dir, output_folder, postfix, 'graph_detailed.svg'));